In [1]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
import numpy as np
import onnx
import onnxruntime as ort
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import random
from PIL import Image

import sys
sys.path.insert(0, "../mlflow-scripts")
from model import get_depthpro_model, LightningModel
from dataloader import get_dataloaders, Urban100Dataset, collate_fn

/Users/xiaoyanouyang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
model = LightningModel(get_depthpro_model(32))
state_dict = torch.load("../model_weights_10.pth", map_location="cpu")
model.load_state_dict(state_dict)

onnx_model_path = "./converted_onn_model_weights.onnx"
# dummy input - used to clarify the input shape
dummy_input = torch.randn(1, 3, 224, 224)  
torch.onnx.export(model, dummy_input, onnx_model_path,
                  export_params=True, opset_version=20,
                  do_constant_folding=True, input_names=['input'],
                  output_names=['output'], dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

print(f"ONNX model saved to {onnx_model_path}")

onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

Some weights of DepthProForDepthEstimation were not initialized from the model checkpoint at geetu040/DepthPro and are newly initialized: ['depth_pro.encoder.feature_projection.projections.4.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.3.2.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.4.3.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.5.4.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.6.0.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.6.1.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.6.2.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.6.3.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.6.4.weight', 'depth_pro.encoder.feature_upsample.upsample_blocks.6.5.weight', 'fusion_stage.layers.4.deconv.weight', 'fusion_stage.layers.5.projection.bias', 'fusion_stage.layers.5.projection.weight', 'fusion_stage.layers.5.residual_layer1.convolution1.bias', 'fusion_stage.layers.5.residual_l

In [ ]:
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
ort_session.get_providers()

In [ ]:
model_size = os.path.getsize(onnx_model_path) 
print(f"Model Size on Disk: {model_size/ (1e6) :.2f} MB")